# util-cos

This component provides COS utility functions (e.g. creating a bucket, listing contents of a bucket)

In [ ]:
import os

#os.environ['create_image']='True'
os.environ['repository']='docker.io/romeokienzler'
os.environ['version']='0.2'
os.environ['name']='claimed-util-cos'

In [ ]:
if bool(os.environ.get('create_image',False)):
    docker_file="""
    FROM debian
    RUN apt update && apt install -y python3-pip procps default-jre
    RUN pip install ipython nbformat aiobotocore botocore s3fs
    ADD util-cos.ipynb .
    """
    with open("Dockerfile", "w") as text_file:
        text_file.write(docker_file)

    !docker build -t `echo $name`:`echo $version` .
    !docker tag `echo $name`:`echo $version` `echo $repository`/`echo $name`:`echo $version`
    !docker push `echo $repository`/`echo $name`:`echo $version`
    !rm Dockerfile
else: 
    None

In [ ]:
import logging
import os
import re
import s3fs
import sys

In [ ]:
# access key id
access_key_id = os.environ.get('access_key_id')

# secret access key
secret_access_key = os.environ.get('secret_access_key')

# cos/s3 endpoint
endpoint = os.environ.get('endpoint')

# cos bucket name
bucket_name = os.environ.get('bucket_name')

# operation
operation = os.environ.get('operation')

In [ ]:
parameters = list(
    map(lambda s: re.sub('$', '"', s),
        map(
            lambda s: s.replace('=', '="'),
            filter(
                lambda s: s.find('=') > -1 and bool(re.match(r'[A-Za-z0-9_]*=[.\/A-Za-z0-9]*', s)),
                sys.argv
            )
    )))

for parameter in parameters:
    logging.debug('Parameter: ' + parameter)
    exec(parameter)

In [ ]:
s3 = s3fs.S3FileSystem(
    anon=False,
    key=access_key_id,
    secret=secret_access_key,
    client_kwargs={'endpoint_url': endpoint}
)

if operation == 'mkbucket':
    s3.mkdir(bucket_name)
else:
    print(f'operation unkonwn {operation}')